# Download Model Llama 3 untuk RAG Project

Notebook ini berisi berbagai cara untuk mendownload model Llama 3 yang akan digunakan dalam proyek RAG (Retrieval Augmented Generation) untuk mencari judul skripsi yang cocok.

## Metode Download yang Tersedia:
1. **Hugging Face Transformers** - Cara termudah dan terintegrasi
2. **Hugging Face Hub** - Download langsung dari repository
3. **Ollama** - Untuk penggunaan lokal yang mudah

## Requirements:
- Python 3.8+
- Koneksi internet yang stabil
- Storage minimal 15GB untuk model

## 1. Install Dependencies

Jalankan cell berikut untuk menginstall semua dependencies yang diperlukan:

In [ ]:
# Install required packages
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cpu
!pip install transformers accelerate bitsandbytes
!pip install huggingface_hub
!pip install tqdm requests

## 2. Import Libraries

In [ ]:
import os
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
from huggingface_hub import hf_hub_download, snapshot_download
import requests
from tqdm import tqdm
import json
from pathlib import Path

print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"CUDA version: {torch.version.cuda}")
    print(f"GPU count: {torch.cuda.device_count()}")

## 3. Setup Directory dan Konfigurasi

In [ ]:
# Setup directories
MODEL_DIR = "./models"
LLAMA3_DIR = os.path.join(MODEL_DIR, "llama3")

# Create directories if they don't exist
os.makedirs(MODEL_DIR, exist_ok=True)
os.makedirs(LLAMA3_DIR, exist_ok=True)

print(f"Model directory: {MODEL_DIR}")
print(f"Llama3 directory: {LLAMA3_DIR}")

# Available Llama 3 models
AVAILABLE_MODELS = {
    "llama3-8b-instruct": "meta-llama/Meta-Llama-3-8B-Instruct",
    "llama3-8b": "meta-llama/Meta-Llama-3-8B",
    "llama3-70b-instruct": "meta-llama/Meta-Llama-3-70B-Instruct",
    "llama3-70b": "meta-llama/Meta-Llama-3-70B"
}

print("\nAvailable models:")
for key, value in AVAILABLE_MODELS.items():
    print(f"  {key}: {value}")

## 4. Metode 1: Download dengan Hugging Face Transformers (Recommended)

Ini adalah cara termudah untuk mendownload dan menggunakan model Llama 3. Model akan otomatis didownload saat pertama kali digunakan.

In [ ]:
# Pilih model yang ingin didownload
# Untuk RAG project, disarankan menggunakan llama3-8b-instruct
selected_model = "llama3-8b-instruct"
model_name = AVAILABLE_MODELS[selected_model]

print(f"Downloading model: {model_name}")
print("Note: Proses ini membutuhkan waktu cukup lama dan koneksi internet yang stabil")

try:
    # Download tokenizer
    print("\n1. Downloading tokenizer...")
    tokenizer = AutoTokenizer.from_pretrained(
        model_name,
        cache_dir=LLAMA3_DIR,
        trust_remote_code=True
    )
    print("✓ Tokenizer downloaded successfully")
    
    # Download model (ini akan memakan waktu lama)
    print("\n2. Downloading model (this may take a very long time...)")
    model = AutoModelForCausalLM.from_pretrained(
        model_name,
        cache_dir=LLAMA3_DIR,
        torch_dtype=torch.float16 if torch.cuda.is_available() else torch.float32,
        device_map="auto" if torch.cuda.is_available() else None,
        trust_remote_code=True,
        low_cpu_mem_usage=True
    )
    print("✓ Model downloaded successfully")
    
    # Test model
    print("\n3. Testing model...")
    test_prompt = "Hello, how are you?"
    inputs = tokenizer(test_prompt, return_tensors="pt")
    
    with torch.no_grad():
        outputs = model.generate(
            inputs.input_ids,
            max_length=50,
            num_return_sequences=1,
            temperature=0.7,
            do_sample=True,
            pad_token_id=tokenizer.eos_token_id
        )
    
    response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    print(f"Test output: {response}")
    print("\n✓ Model is working correctly!")
    
except Exception as e:
    print(f"❌ Error downloading model: {str(e)}")
    print("\nPossible solutions:")
    print("1. Make sure you have enough disk space (15GB+)")
    print("2. Check your internet connection")
    print("3. You might need to request access to Llama 3 models on Hugging Face")
    print("4. Try using a smaller model or Ollama method below")

## 5. Metode 2: Download dengan Hugging Face Hub

Metode ini memberikan kontrol lebih detail atas proses download.

In [ ]:
def download_model_with_hub(model_name, local_dir):
    """
    Download model menggunakan huggingface_hub
    """
    try:
        print(f"Downloading {model_name} to {local_dir}")
        
        # Download entire model repository
        snapshot_download(
            repo_id=model_name,
            local_dir=local_dir,
            local_dir_use_symlinks=False,
            resume_download=True
        )
        
        print(f"✓ Model downloaded successfully to {local_dir}")
        return True
        
    except Exception as e:
        print(f"❌ Error: {str(e)}")
        return False

# Uncomment the line below to download using this method
# download_model_with_hub(AVAILABLE_MODELS["llama3-8b-instruct"], os.path.join(LLAMA3_DIR, "hf_download"))

## 6. Metode 3: Setup Ollama (Alternative)

Ollama adalah cara mudah untuk menjalankan model lokal. Ini adalah alternatif yang bagus jika method di atas tidak berhasil.

In [ ]:
def setup_ollama():
    """
    Setup dan download Llama 3 menggunakan Ollama
    """
    print("Setting up Ollama...")
    
    # Install Ollama (Linux/Mac)
    print("\n1. Installing Ollama...")
    try:
        # Download and install Ollama
        os.system("curl -fsSL https://ollama.ai/install.sh | sh")
        print("✓ Ollama installed")
        
        # Pull Llama 3 model
        print("\n2. Pulling Llama 3 model...")
        result = os.system("ollama pull llama3")
        
        if result == 0:
            print("✓ Llama 3 model downloaded with Ollama")
            
            # Test the model
            print("\n3. Testing model...")
            os.system('ollama run llama3 "Hello, how are you?"')
            
        else:
            print("❌ Failed to download model")
            
    except Exception as e:
        print(f"❌ Error setting up Ollama: {str(e)}")
        print("\nManual installation:")
        print("1. Visit https://ollama.ai/")
        print("2. Download and install Ollama for your OS")
        print("3. Run: ollama pull llama3")

# Uncomment to setup Ollama
# setup_ollama()

## 7. Verify Model Installation

In [ ]:
def verify_model_installation():
    """
    Verify bahwa model sudah terinstall dengan benar
    """
    print("Verifying model installation...")
    
    # Check local directories
    print(f"\n1. Checking directory: {LLAMA3_DIR}")
    if os.path.exists(LLAMA3_DIR):
        files = os.listdir(LLAMA3_DIR)
        print(f"   Found {len(files)} items")
        for item in files[:5]:  # Show first 5 items
            print(f"   - {item}")
        if len(files) > 5:
            print(f"   ... and {len(files) - 5} more items")
    else:
        print("   Directory not found")
    
    # Check Hugging Face cache
    print("\n2. Checking Hugging Face cache...")
    hf_cache = os.path.expanduser("~/.cache/huggingface")
    if os.path.exists(hf_cache):
        print(f"   HF Cache directory exists: {hf_cache}")
        # Look for Llama models
        transformers_cache = os.path.join(hf_cache, "transformers")
        if os.path.exists(transformers_cache):
            cached_models = [d for d in os.listdir(transformers_cache) if "llama" in d.lower()]
            print(f"   Found {len(cached_models)} Llama-related cached models")
            for model in cached_models[:3]:
                print(f"   - {model}")
    
    # Check available disk space
    print("\n3. Checking available disk space...")
    import shutil
    total, used, free = shutil.disk_usage(".")
    print(f"   Total: {total // (1024**3)} GB")
    print(f"   Used: {used // (1024**3)} GB")
    print(f"   Free: {free // (1024**3)} GB")
    
    if free < 15 * 1024**3:  # Less than 15GB
        print("   ⚠️  Warning: Low disk space. Consider freeing up space before downloading large models.")
    else:
        print("   ✓ Sufficient disk space available")

verify_model_installation()

## 8. Simple RAG Example

Contoh sederhana menggunakan model yang sudah didownload untuk RAG task.

In [ ]:
def simple_rag_example():
    """
    Contoh sederhana penggunaan Llama 3 untuk mencari judul skripsi
    """
    # Sample thesis topics database
    thesis_topics = [
        "Implementasi Machine Learning untuk Prediksi Harga Saham",
        "Sistem Informasi Manajemen Perpustakaan Berbasis Web",
        "Analisis Sentimen Media Sosial Menggunakan Natural Language Processing",
        "Aplikasi Mobile untuk Monitoring Kesehatan Berbasis IoT",
        "Pengembangan Chatbot Customer Service Menggunakan Deep Learning"
    ]
    
    # User query
    user_interest = "machine learning dan artificial intelligence"
    
    # Simple retrieval (in real RAG, this would be more sophisticated)
    relevant_topics = [topic for topic in thesis_topics if any(keyword in topic.lower() for keyword in ["machine", "learning", "ai", "nlp", "deep"])]
    
    # Create prompt for Llama 3
    context = "\n".join([f"- {topic}" for topic in relevant_topics])
    
    prompt = f"""Berdasarkan topik-topik skripsi berikut:
{context}

Dan minat mahasiswa dalam: {user_interest}

Rekomendasikan judul skripsi yang paling sesuai dan berikan alasannya:"""
    
    print("=== Contoh RAG untuk Rekomendasi Judul Skripsi ===")
    print(f"\nPrompt:")
    print(prompt)
    print("\n" + "="*50)
    
    # Note: This would use the actual model if loaded
    print("\nNote: Untuk menggunakan model yang sudah didownload, load model terlebih dahulu dengan code di cell sebelumnya.")
    print("Kemudian generate response menggunakan model.generate() dengan prompt di atas.")

simple_rag_example()

## 9. Troubleshooting dan Tips

### Common Issues:

1. **Access Denied Error**
   - Anda perlu request access ke Meta Llama 3 models di Hugging Face
   - Visit: https://huggingface.co/meta-llama/Meta-Llama-3-8B-Instruct
   - Login dan request access

2. **Out of Memory Error**
   - Gunakan `torch.float16` atau `torch.bfloat16`
   - Enable `low_cpu_mem_usage=True`
   - Gunakan model 8B instead of 70B

3. **Slow Download**
   - Use `resume_download=True`
   - Check internet connection
   - Try downloading during off-peak hours

### Tips untuk Optimization:

1. **Untuk Production**: Gunakan quantized models (4bit/8bit)
2. **Untuk Development**: Gunakan Ollama untuk kemudahan
3. **Untuk Fine-tuning**: Download full precision models

### Next Steps:

1. Implement proper vector database untuk RAG
2. Add embedding models untuk document retrieval
3. Create API wrapper untuk model
4. Add evaluation metrics untuk RAG performance

## 10. Save Configuration

In [ ]:
# Save configuration untuk future use
config = {
    "model_dir": MODEL_DIR,
    "llama3_dir": LLAMA3_DIR,
    "selected_model": selected_model,
    "model_name": AVAILABLE_MODELS.get(selected_model, ""),
    "available_models": AVAILABLE_MODELS,
    "setup_date": str(pd.Timestamp.now())
}

config_file = os.path.join(MODEL_DIR, "llama3_config.json")
with open(config_file, "w") as f:
    json.dump(config, f, indent=2)

print(f"Configuration saved to: {config_file}")
print("\nConfiguration:")
print(json.dumps(config, indent=2))